In [18]:
##### -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import anndata
import scprep
import scanpy as sc
import sklearn
import tempfile
from os import path

import os
import sys
import scipy
from scipy import sparse
from torch.nn.utils.rnn import pad_sequence
import torch
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import glob
from cca_loss import cca_loss
from FC_VAE import FC_VAE


def load_binary_file(file_name, dimension):
    fid_lab = open(file_name, 'rb')
    features = np.fromfile(fid_lab, dtype=np.float32)
    fid_lab.close()
    assert features.size % float(dimension) == 0.0, 'specified dimension %s not compatible with data' % (dimension)
    features = features[:(dimension * (features.size // dimension))]
    features = features.reshape((-1, dimension))
    return features

def load_binary_file_frame(self, file_name, dimension):
    fid_lab = open(file_name, 'rb')
    features = numpy.fromfile(fid_lab, dtype=numpy.float32)
    fid_lab.close()
    assert features.size % float(dimension) == 0.0, 'specified dimension %s not compatible with data' % (dimension)
    frame_number = features.size // dimension
    features = features[:(dimension * frame_number)]
    features = features.reshape((-1, dimension))

    return features, frame_number



dir_phonePath = "/home/rania/Documents/workspace/tools/merlin/output"
dir_PPGpath = "/home/rania/Documents/workspace/IntraSpkVC/data_fr/ppg/FFR0009"
phone_data = []
PPG_data = []

PPG_lengths = []
phone_lengths = []


def load_data(dir_path, x):
    
    file_list = glob.glob(dir_path + '/*.'+x+'')
    
    for idx, s in enumerate(file_list):
        if x == 'flab':
            data =torch.tensor(load_binary_file(s, 49))
#             print(data.shape)
#           print(len(data[:,0]))
            phone_data.append(data)
            phone_lengths.append(len(data))
        else:
            data = np.load(s)
            PPG_data.append(data)
            PPG_lengths.append(len(data))

    return phone_data, phone_lengths, PPG_data, PPG_lengths

phone_data, phone_lengths, _, _ = load_data(dir_phonePath, 'flab')
# print("phone_data {} {}".format(  len(phone_data), len(phone_data[1])  )    )

class Dataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.data = pad_sequence(data, batch_first=True, padding_value=0)
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx].clone().detach().requires_grad_(True)
        x = torch.unsqueeze(x, 0)
        return x  

# _, _, PPG_data, PPG_lengths = load_data(dir_phonePath, 'npy')

padded_phone = Dataset(phone_data)
iterator = iter(padded_phone)

for i, item in enumerate(train_loader):
    print(item.shape)


torch.Size([32, 1, 2386, 49])


In [2]:
hyper = {
    "nEpochs":100,
    "dim_rna":2386,
    "dim_atac":2386,
    "n_latent":32,
    "layer_sizes":[256, 64],
    "add_hinge":False,
    "lamb_match":1,
    "lamb_nn":1.5,
    "lamb_hinge":10,
    "train_batch":256,
    "test_batch": 512,
    "lr": 1e-3,
    "clip_grad": 1,
    "checkpoint_path": './checkpoint/best_model_dcca_hinge.pt',
    "device": "cuda" if torch.cuda.is_available() else "cpu",
}

In [3]:
def recon_loss(x, x_recon, y, y_recon):
    MSE = nn.MSELoss()
    x_loss = MSE(x, x_recon)
    y_loss = MSE(y, y_recon)
    total_loss = x_loss + y_loss
    return total_loss

In [4]:
class Encoder(nn.Module):
    def __init__(self, n_input, n_latent, layer_sizes):
        super(Encoder, self).__init__()
        self.n_input = n_input
        self.n_latent = n_latent
        self.layer_sizes = [n_input] + layer_sizes + [n_latent]
        self.encoder_layers = []
        
        for idx in range(len(self.layer_sizes) - 1):
            fc1 = nn.Linear(self.layer_sizes[idx], self.layer_sizes[idx + 1])
            nn.init.xavier_uniform_(fc1.weight)
            self.encoder_layers.append(fc1)
            bn1 = nn.BatchNorm1d(self.layer_sizes[idx + 1])
            self.encoder_layers.append(bn1)
            act1 = nn.PReLU()
            self.encoder_layers.append(act1)
            
        self.encoder = nn.Sequential(*self.encoder_layers)
        
    def forward(self, x):
        return(self.encoder(x))

In [5]:
class DCCA(nn.Module):
    def __init__(
        self, 
        n_input1, 
        n_input2, 
        layer_sizes1, 
        layer_sizes2, 
        n_out, 
        use_all_singvals=False, 
        device=torch.device("cpu"), 
        use_decode = False, 
        seed=182822,
    ):
        super(DCCA, self).__init__()
        torch.manual_seed(seed)
        self.Net1 = Encoder(n_input1, n_latent=n_out, layer_sizes=layer_sizes1)
        self.Net2 = Encoder(n_input2, n_latent=n_out, layer_sizes=layer_sizes2)
        self.cca_loss = cca_loss(out_dim=n_out, use_all_singvals=use_all_singvals, device=device).loss
        self.recon_loss = recon_loss
        
        self.device = device
        self.use_decode = use_decode
        
    def forward(self, x1, x2):
        z1 = self.Net1(x1)
        embed1 = F.normalize(z1, dim=1, p=2)
        z2 = self.Net2(x2)
        embed2 = F.normalize(z2, dim=1, p=2)
        return embed1, embed2


In [8]:
def get_data_loaders(train_data_filtered, test_data_filtered, train_data_raw, test_data_raw):
    train_set = trainDataX
    test_set = valDataX
    
    #load data loader
    train_loader = DataLoader(
        train_set, 
        batch_size=hyper["train_batch"], 
        drop_last=False, 
        shuffle=True,
    )
    test_loader = DataLoader(
        test_set, 
        batch_size=hyper["test_batch"], 
        drop_last=False, 
        shuffle=False,
    )
    return train_loader, test_loader


In [11]:
trainDataX, valDataX,trainDataY, valDataY = train_test_split([data for data in padded_phone ],[data for data in padded_phone ],test_size=0.2,random_state=41,shuffle=False,stratify=None)
# print("size of the training dataset {}".format(len(trainDataX)), trainDataX[2].shape)

train_loader, test_loader = get_data_loaders(
        trainDataX, 
        valDataX,
        trainDataX,
        valDataX,
    )

print(train_loader, test_loader)
for i, item in enumerate(train_loader):
    print(item.shape)

#load checkpoint
checkpoint = None
if path.exists(hyper["checkpoint_path"]):
    checkpoint = torch.load(hyper["checkpoint_path"])

my_device = 'cpu'    
#load basic models
dcca_model = DCCA(
    n_input1=hyper["dim_rna"], 
    n_input2=hyper["dim_atac"], 
    layer_sizes1=hyper["layer_sizes"], 
    layer_sizes2=hyper["layer_sizes"], 
    n_out=hyper["n_latent"],
    use_all_singvals=False,
    device=torch.device(my_device), 
    use_decode = False,
)


if checkpoint != None:
    dcca_model.load_state_dict(checkpoint["netPhone_state_dict"])

if torch.cuda.is_available():
    print("using GPU")
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
dcca_model.to(device)

#set up optimizer
dcca_opt = optim.Adam(list(dcca_model.parameters()), lr=hyper["lr"])
dcca_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    dcca_opt, 
    patience=10,
    factor=0.5,
    threshold=0.3, 
    threshold_mode="abs",
    min_lr=1e-5,
)

#set up device 
device=hyper["device"]
dcca_model.to(device)



#training step
for epoch in range(hyper["nEpochs"]):
    train_losses = []
    for idx, (phone_data) in enumerate(train_loader):
#         phone_data.zero_grad()

        phone_data = Variable(phone_data).to(device)
        
        embed_1, embed_2 = dcca_model(phone_data, phone_data)

        #calculate train loss
        train_loss = dcca_model.cca_loss(embed_1, embed_2)
        if hyper["add_hinge"]:
            hinge_loss = StructureHingeLoss(
                margin=0.25, 
                max_val=1e6, 
                lamb_match=hyper["lamb_match"], 
                lamb_nn=hyper["lamb_nn"],
                device=device,
            )
            h_loss = hinge_loss(embed_rna, embed_atac, nn_indices)
            #train_loss = train_loss + h_loss
            train_loss += hyper['lamb_hinge'] * h_loss
            #print(f"train loss: {train_loss}")
            #print(f"hinge loss: {h_loss}")
            #print(f"train loss with hinge: {train_loss}")
        train_loss.backward()
        #nn.utils.clip_grad_norm_(toy_model.parameters(), max_norm=hyper["clip_grad"])
        dcca_opt.step()
        dcca_scheduler.step(train_loss)
        train_losses.append(train_loss.item())
    avg_train_loss = np.mean(train_losses)
    if epoch % 5 == 0:
        print("Epoch: " + str(epoch) + ", train loss: " + str(avg_train_loss))


<torch.utils.data.dataloader.DataLoader object at 0x7f33443cbc10> <torch.utils.data.dataloader.DataLoader object at 0x7f33443cb130>
torch.Size([32, 1, 2386, 49])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (76352x49 and 2386x256)